In [66]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [73]:
# Load the csv exported in Part I to a DataFrame
vacation_df = pd.read_csv("city_weather_test.csv")
vacation_df.drop(columns="Unnamed: 0", inplace=True)
vacation_df.head()

,Lat,Lng,City,Country,Temperature,Humidity,Cloud cover,Wind speed,Date
0,51.917854,-89.269802,geraldton,ca,18.23,77.0,90.0,3.60,2021-07-22 08:52:30
1,-28.899229,30.521664,greytown,za,11.74,39.0,19.0,1.81,2021-07-22 08:53:34
2,37.873070,96.718583,yumen,cn,33.84,24.0,56.0,9.25,2021-07-22 08:53:35
3,-82.124047,-133.862385,rikitea,pf,22.30,71.0,0.0,6.69,2021-07-22 08:53:36
4,56.284596,-146.129518,palmer,us,16.92,91.0,1.0,0.00,2021-07-22 08:53:37


 Humidity Heatmap

In [77]:
# Configure gmaps
gmaps.configure(api_key=g_key)
# Lat and Lng as locations and Humidity as the weight.Add Heatmap layer to map
locations = vacation_df[["Lat", "Lng"]]
humidity = vacation_df["Humidity"]
map_1 = gmaps.figure(zoom_level=2, center=[20,50])
heatmap_1 = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=350, point_radius=6, dissipating=False)
map_1.add_layer(heatmap_1)
map_1

Figure(layout=FigureLayout(height='420px'))

In [79]:
#  Create new DataFrame fitting weather criteria
ideal_vacation = vacation_df[(vacation_df["Temperature"]>=25) & (vacation_df["Temperature"]<=40)]
ideal_vacation = ideal_vacation[ideal_vacation["Wind speed"]<10]
ideal_vacation = ideal_vacation[ideal_vacation["Cloud cover"]<20]
ideal_vacation.head()

,Lat,Lng,City,Country,Temperature,Humidity,Cloud cover,Wind speed,Date
8,25.424390,23.809887,jalu,ly,31.28,42.0,9.0,4.00,2021-07-22 08:53:40
11,31.309230,140.513360,shimoda,jp,28.20,78.0,1.0,2.40,2021-07-22 08:53:45
20,-0.302607,30.269913,bushenyi,ug,26.68,28.0,4.0,4.09,2021-07-22 08:52:23
23,42.786456,90.235451,urumqi,cn,30.53,25.0,0.0,6.00,2021-07-22 08:53:56
51,32.018131,96.622965,tezu,in,36.41,48.0,18.0,0.70,2021-07-22 08:54:15


In [ ]:
# Hit the Google Places API for each city's coordinates.


In [20]:
# Hotel Map
# Store into variable named hotel_df.Add a "Hotel Name" column to the DataFrame.
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

param = {
    "key":g_key,
    "radius":5000,
    "type":"lodging",
    "location": "-31.9523, 115.8613"
}
# param["location"] = f"{lat},{lng}"
ret = requests.get(url, params=param).json()


In [21]:
ret["results"][0]

{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': -31.95436680000001, 'lng': 115.8529193},
  'viewport': {'northeast': {'lat': -31.9529586697085,
    'lng': 115.8541703802915},
   'southwest': {'lat': -31.9556566302915, 'lng': 115.8514724197085}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
 'name': 'Parmelia Hilton Perth',
 'photos': [{'height': 3000,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/105287034285661705751">Parmelia Hilton Perth</a>'],
   'photo_reference': 'Aap_uEDgiXPLh0tocbXl3MXYBGMlpm1UtMPyiS99k0z87WlKNprl71Wt8qG6YrU-rDHy3de3EYDWl038eWw36CTIF-wQX9gwq1cuNundYXX8tw2nyywxoGn4xjxO4XiDL6qtxLvEkyLAEVQozmH3XeTeQWFzBAp-3kRDw3RwuahJICpbvd6f',
   'width': 4507}],
 'place_id': 'ChIJQdM1JtW6MioRxAJto5QX2Qw',
 'plus_code': {'compound_code': '2VW3+75 Perth WA, Australia',
  'global_code': '4PWQ2VW3+75'},
 'rating': 4.2,
 'reference': 'ChIJQdM1JtW6MioRxAJto5QX2Qw',
 'scope': 'GOOGLE',
 'types': [

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure